In [1]:
import pandas as pd
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
import re
from collections import Counter
import os

In [2]:
fastq_folder = "dep fastq files"
fastq_files = [os.path.join(fastq_folder, file) for file in os.listdir(fastq_folder) if file.endswith(".fastq")]

all_genera = []

for fastq_file in fastq_files:
    ids = []
    sequences = []

    with open(fastq_file, "r") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            ids.append(record.id)
            sequences.append(str(record.seq))


    df1 = pd.DataFrame({'ID': ids, 'Sequence': sequences})

    genomic_sequence = df1['Sequence'][0][:1000]
    result_handle = NCBIWWW.qblast("blastn", "nt", genomic_sequence)
    blast_records = NCBIXML.parse(result_handle)

    genera = []

    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                if hsp.expect < 0.05:
                    order_match = re.search(r"Uncultured (\b[A-Z][a-z]+(?:ales|aceae|ota|etes)\b)", alignment.title)
                    match = re.search(r"Uncultured ([A-Z][a-z]+) sp", alignment.title)
                    general_match = re.search(r"\b([A-Z][a-z]+)\b(?!\s*(bacterium|organism|RNA|DNA))", 
                                              alignment.title)
                    unknown_match = re.search(r"Uncultured (bacterium|organism)", alignment.title, re.IGNORECASE)
                    rumen_match = re.search(r"Uncultured rumen bacterium", alignment.title, re.IGNORECASE)

                    if order_match:
                        genera.append(order_match.group(1))
                    elif match:
                        genera.append(match.group(1))
                    elif rumen_match:
                        genera.append("Rumen Bacterium")
                    elif general_match:
                        genera.append(general_match.group(1))
                    elif unknown_match:
                        genera.append("Uncultured Organism")
                    
    genus_counts = Counter(genera)

    df = pd.DataFrame(genus_counts.items(), columns=['Genus', 'Count'])

    df['Relative Abundance'] = df['Count'] / df['Count'].sum()

    df = df.drop(columns=['Count'])

    df = df.sort_values(by='Relative Abundance', ascending=False)
    
    df = df.T

    df.insert(0, 'File Name', os.path.basename(fastq_file))
    
    df = df.reset_index(drop=True)
    
    df.columns = df.iloc[0]
    df = df.drop(0)
    
    df = df.rename(columns={os.path.basename(fastq_file): 'File Name'})
    
    all_genera.append(df)
    
final_df = pd.concat(all_genera, ignore_index=True)


/Users/melinakopischkie/anaconda3/lib/python3.11/site-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request UZ8EE44E013 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(
/Users/melinakopischkie/anaconda3/lib/python3.11/site-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request V0230PF8016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


In [3]:
final_df

,File Name,Bacteroides,Parabacteroides,Phocaeicola,Butyricimonas,Uncultured Organism,Faecalibacterium,Intestinimonas,Streptomyces,Candidatus,...,Emergencia,Adlercreutzia,Pelobacter,Dehalogenimonas,Nitrospira,Bradymonas,Sporolactobacillus,Desulfoscipio,Desulfuromonas,Heliomicrobium
0,ERR10786192.fastq,0.484375,0.265625,0.21875,0.015625,0.015625,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ERR10786176.fastq,NaN,NaN,NaN,NaN,NaN,0.73913,0.065217,0.043478,0.021739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ERR10786189.fastq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ERR10786174.fastq,NaN,NaN,NaN,NaN,NaN,NaN,0.078947,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ERR10786190.fastq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ERR10786169.fastq,0.055556,NaN,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ERR10786194.fastq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ERR10786210.fastq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ERR10786170.fastq,0.921569,0.078431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ERR10786209.fastq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
final_df.to_csv('extra_dep_data.csv')

In [ ]:
# final_df = pd.concat(all_genera, ignore_index=True)
# #final_df = final_df.drop(['File Name'], axis=1)
# final_df

In [ ]:
# dft = final_df.T
# dft = dft.reset_index()
# dft.columns = dft.iloc[0]
# df_final = dft.drop(0).reset_index(drop=True)
# #df_final = df_final.iloc[1:].reset_index(drop=True)
# #df_final = df_final.drop(df_final.columns[0], axis=1)
# df_final = df_final.rename(columns={'Genus': 'File Name'})
# df_final

In [ ]:
# dft.reset_index()
# dft

In [ ]:
# genomic_sequence = df1['Sequence'][0][:1000]
# result_handle = NCBIWWW.qblast("blastn", "nt", genomic_sequence)
# blast_records = NCBIXML.parse(result_handle)

In [ ]:
# genera = []

# for blast_record in blast_records:
#     for alignment in blast_record.alignments:
#         for hsp in alignment.hsps:
#             if hsp.expect < 0.05:
#                 match = re.search(r"\b([A-Z][a-z]+)\b", alignment.title)  # Extract first capitalized word
#                 if match:
#                     genus = match.group(1)
#                     genera.append(genus)
                    
# genus_counts = Counter(genera)

# df = pd.DataFrame(genus_counts.items(), columns=['Genus', 'Count'])

# df['Relative Abundance'] = df['Count'] / df['Count'].sum()

# df = df.drop(columns=['Count'])

# df = df.sort_values(by='Relative Abundance', ascending=False)



In [ ]:
# df = df.rename(columns={'Genus': 'Run ID', 'Relative Abundance': 'file name'})
# df

In [ ]:
# dft = df.T
# dft = dft.reset_index()
# dft.columns = dft.iloc[0]
# df_final = dft.drop(0).reset_index(drop=True)
# df_final

In [ ]:
# all_genera = []
# for fastq_file in os.listdir(fastq_folder):
#     fastq_path = os.path.join(fastq_folder, fastq_file)
#     for path in fastq_path:
#         genera = []

#         for blast_record in blast_records:
#             for alignment in blast_record.alignments:
#                 for hsp in alignment.hsps:
#                     if hsp.expect < 0.05:
#                         match = re.search(r"\b([A-Z][a-z]+)\b", alignment.title)
#                         if match:
#                             genus = match.group(1)
#                             if genus.lower() not in ["uncultured", "bacterium", "organism"]:
#                                 genera.append(genus)

# list_genera = all_genera

# # genus_counts = Counter(genera)

# # df = pd.DataFrame(genus_counts.items(), columns=['Genus', 'Count'])

# # df['Relative Abundance'] = df['Count'] / df['Count'].sum()

# # df = df.sort_values(by='Relative Abundance', ascending=False)

# # df.insert(0, 'Unique Sequence ID', [seq_id for seq_id, _ in all_genera])

# # df

In [ ]:
# all_genera = []

# for index, row in df.iterrows():
#     unique_id = row["ID"]
#     genomic_sequence = row["Sequence"][:1000]


#     result_handle = NCBIWWW.qblast("blastn", "nt", genomic_sequence)
#     blast_records = NCBIXML.parse(result_handle)

#     genera = []
#     for blast_record in blast_records:
#         for alignment in blast_record.alignments:
#             for hsp in alignment.hsps:
#                 if hsp.expect < 0.05:  # Consider only significant matches
#                     match = re.search(r"\b([A-Z][a-z]+)\b", alignment.title)
#                     if match:
#                         genus = match.group(1)
#                         if genus.lower() not in ["uncultured", "bacterium", "organism"]:
#                             genera.append(genus)

# all_genera.append({"ID": unique_id, "Genera": genera})


In [ ]:
# flat_genera = [genus for _, genera in all_genera for genus in genera]
# genus_counts = Counter(flat_genera)

# df_genera = pd.DataFrame(genus_counts.items(), columns=['Genus', 'Count'])

# df_genera['Relative Abundance'] = df_genera['Count'] / df_genera['Count'].sum()

# df_genera = df_genera.sort_values(by='Relative Abundance', ascending=False)

# print(df_genera)